## Project description
- Performing sentiment analysis using BERT model: https://huggingface.co/docs/transformers/model_doc/bert
- Dataset is available here: https://figshare.com/articles/dataset/smile_annotations_final_csv/3187909/2?file=4988956

- This is a simple sample project just to showcase the BERT model.

In [2]:
#Importing all libraries that are needed.
import torch
import pandas as pd
from tqdm.notebook import tqdm

In [15]:
#Importing dataset.
df_smile = pd.read_csv('/content/smileannotationsfinal.csv',
                      names = ['id', 'tweet', 'mood'])
#Setting the 'id' column as the new index for the DataFrame 'df_smile'.
#The 'inplace=True' argument modifies the DataFrame in-place, without creating a new DataFrame.
df_smile.set_index('id', inplace= True)

In [17]:
#Checking the dataframe.
df_smile.head()

,tweet,mood
id,,
611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy


In [18]:
df_smile.mood.value_counts()

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|disgust             2
sad|angry               2
sad|disgust|angry       1
Name: mood, dtype: int64

In [19]:
#Removing all ambiguous mood categories.
df_smile = df_smile[~df_smile.mood.str.contains('\|') & (df_smile.mood != 'nocode')]

In [20]:
df_smile.head()

,tweet,mood
id,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy


In [21]:
df_smile.mood.value_counts()

happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: mood, dtype: int64

In [22]:
#Making labels adding them to the dictionary.
labels = df_smile.mood.unique()
label_dict = {}
for index, labels in enumerate(labels):
  label_dict[labels] = index


In [23]:
label_dict

{'happy': 0,
 'not-relevant': 1,
 'angry': 2,
 'disgust': 3,
 'sad': 4,
 'surprise': 5}

In [24]:
df_smile['mood_label'] = df_smile['mood'].map(label_dict)

In [25]:
df_smile.head(10)

,tweet,mood,mood_label
id,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0
614499696015503361,Lucky @FitzMuseum_UK! Good luck @MirandaStearn...,happy,0
613601881441570816,Yr 9 art students are off to the @britishmuseu...,happy,0
613696526297210880,@RAMMuseum Please vote for us as @sainsbury #s...,not-relevant,1
610746718641102848,#AskTheGallery Have you got plans to privatise...,not-relevant,1


## Training and Validation

In [26]:
#Importing the train_test_split function from the sklearn.model_selection module.
from sklearn.model_selection import train_test_split

In [27]:
#splitting our dataset
X_train, X_val, y_train, y_val = train_test_split(
    df_smile.index.values,
    df_smile.mood_label.values,
    test_size =0.15,
    random_state= 17,
    stratify = df_smile.mood_label.values
    )

In [28]:
df_smile['data_type'] = ['not_set']*df_smile.shape[0]

In [29]:
df_smile.loc[X_train, 'data_type'] = 'train'
df_smile.loc[X_val, 'data_type'] = 'val'

In [30]:
df_smile.head()

,tweet,mood,mood_label,data_type
id,,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0,train
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0,train
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0,train
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0,train
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0,train


In [31]:
df_smile.groupby(['mood','mood_label', 'data_type']).count()

tweet
mood         mood_label data_type       
angry        2          train         48
                        val            9
disgust      3          train          5
                        val            1
happy        0          train        966
                        val          171
not-relevant 1          train        182
                        val           32
sad          4          train         27
                        val            5
surprise     5          train         30
                        val            5

## Loading tokenizer and encoding

In [32]:
!pip install transformers

In [33]:
#Imporingt the BertTokenizer class from the transformers module.
#Importing the TensorDataset class from the torch.utils.data module.

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [34]:
#Creating a tokenizer object using the BERT model with the 'bert-base-uncased' configuration.
#The tokenizer is responsible for converting text inputs into numerical tokens that can be understood by the model.
#The 'do_lower_case=True' argument ensures that all text is converted to lowercase before tokenization,
#Which is important for models that were trained on uncased text.
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


In [35]:
#Encoding training data using the tokenizer's batch_encode_plus method.
#The tweet values from the DataFrame 'df_smile' where the data type is 'train' are passed as input.
#Special tokens are added, attention masks are returned, and the sequences are padded to the maximum length of 256.
#The encoded data is returned as PyTorch tensors.
encoded_data_train = tokenizer.batch_encode_plus(
    df_smile[df_smile.data_type=='train'].tweet.values,
    add_special_tokens = True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [36]:
#Encoding the validation data using the tokenizer's batch_encode_plus method.
#The tweet values from the DataFrame 'df_smile' where the data type is 'val' are passed as input.
#Special tokens are added, attention masks are returned, and the sequences are padded to the maximum length of 256.
#The encoded data is returned as PyTorch tensors.
encoded_data_val = tokenizer.batch_encode_plus(
    df_smile[df_smile.data_type=='val'].tweet.values,
    add_special_tokens = True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [40]:
#Extracting input IDs, attention masks, and labels for the training set.
input_ids_train= encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df_smile[df_smile.data_type=='train'].mood_label.values)

#Extracting input IDs, attention masks, and labels for the validation.
input_ids_val= encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df_smile[df_smile.data_type=='val'].mood_label.values)

In [41]:
#Creating 2 different datasets:one for training and the second for validation
dataset_train = TensorDataset(input_ids_train,attention_masks_train,labels_train)
dataset_val = TensorDataset(input_ids_val,attention_masks_val, labels_val)

## Settingup BERT pretrained model


In [44]:
#BertForSequenceClassification is a pre-trained BERT model fine-tuned for sequence classification tasks.
from transformers import BertForSequenceClassification

In [45]:
#Loading a pre-trained BERT model for sequence classification using the 'bert-base-uncased' configuration.
#The number of labels is set to the length of the label dictionary, which determines the number of output classes.
#The 'output_attentions' and 'output_hidden_states' parameters are set to False to disable the output of attention weights and hidden states.

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = len(label_dict),output_attentions=False, output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

## Creating data loaders


In [50]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [52]:
batch_size = 4
dataloader_train = DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size
)

dataloader_val = DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=32
)

## Optimizer and Schdeulder


In [54]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [55]:
optimizer = AdamW(model.parameters(), lr=1e-5, eps=1e-8)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [56]:
epochs=15
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(dataloader_train)*epochs)

## Defining performance metrics


In [58]:
import numpy as np
from sklearn.metrics import f1_score

In [59]:
def f1_score(preds, lables):
  preds_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = lebels.flatten()
  return f1_score(labels_flat, preds_flat,avarage='weighted')

In [60]:
def accuracy_per_class(preds, labels):
  label_dict_inversed = {v: k for k, v in label_dict.items()}
  preds_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = lebels.flatten()

  for lael in np.unique(labels_flat):
    y_preds = preds_flat[labels_flat==label]
    y_true = labels_flat[labels_flat==label]
    print(f'Class:{label_dict_inversed[label]}')
    print(f'Accuracy:{len(y_preds[y_preds==label])}/{len(y_true)}\n')

## Creating Training loop

In [61]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [64]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

cpu


In [65]:
def evaluate(dataloader_val):

    model.eval()

    loss_val_total = 0
    predictions, true_vals = [], []

    for batch in dataloader_val:

        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():
            outputs = model(**inputs)

        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)

    loss_val_avg = loss_val_total/len(dataloader_val)

    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)

    return loss_val_avg, predictions, true_vals

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    model.train()
    loss_train_total = 0
    progress_bar = tqdm(dataloader_train, desc='Epoch{:id}'.format(epoch), leave=False,disable = False)
    for batch in progress_bar:
      model.zero_grad()
      batch = tuple(b.to(device) for b in batch)
      inputs = {
          'input_ids':      batch[0],
          'attention_mask': batch[1],
          'labels':         batch[2]
      }

      outputs = model(**inputs)

      loss = outputs[0]
      loss_train_total += loss.item()
      loss.backward()

      torch.nn_utils.clip_grad_norm_(model.parameters(),1.0)

      optimizer.step()
      scheduler.step()

      progress_bar.set_postfix({'training_loss':'{:.3f}'.format(loss.item()/len(batch))})

  torch.save(model.state_dict(), f'Models/BERT_ft_epoch{epoch}.model')
  tqdm.write('\nEpoch {epoch}')
  loss_train_avg = loss_train_total/len(dataloader)
  tqdm.write(f'Training loss:{loss_train_avg}')

  val_loss, predictions, true_vals = evaluate(dataloader_val)
  val_f1 = f1_score_funct(predictions, true_vals)
  tqdm.write(f'Validation loss: {val_loss}')
  tqdm.write(f'F1 Score(weighted):{val_f1}')

In [ ]:
#Loop through the specified number of epochs.
for epoch in tqdm(range(1, epochs+1)):
    #the model to training mode.
    model.train()
    loss_train_total = 0

    #Creating a progress bar for the training dataloader.
    progress_bar = tqdm(dataloader_train, desc='Epoch {:d}'.format(epoch), leave=False, disable=False)

    #Iterate over each batch in the training dataloader.
    for batch in progress_bar:
        # Zero the gradients.
        model.zero_grad()

        # Move the batch tensors to the device.
        batch = tuple(b.to(device) for b in batch)

        # Prepare the inputs for the model.
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2]
        }

        #Forward pass through the model.
        outputs = model(**inputs)

        #Compute the loss.
        loss = outputs[0]
        loss_train_total += loss.item()

        #Backpropagation and gradient clipping.
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        #Update the optimizer and scheduler.
        optimizer.step()
        scheduler.step()

        #Update the progress bar with the current training loss.
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item() / len(batch))})

    #Save the model at the end of each epoch.
    torch.save(model.state_dict(), f'Models/BERT_ft_epoch{epoch}.model')

    #Print epoch information.
    tqdm.write('\nEpoch {:d}'.format(epoch))
    loss_train_avg = loss_train_total / len(dataloader)
    tqdm.write(f'Training loss: {loss_train_avg}')

    #Evaluate the model on the validation set.
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_funct(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (weighted): {val_f1}')

  0%|          | 0/15 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/315 [00:00<?, ?it/s]